# Package functions presentation (Krótkie wyjaśnienie funkcji pakietu statystycznego)
Autor: Konstanty Kłosiewicz student MIMUW

### Ładowanie danych i krótkie podsumowanie

In [2]:
from alco_analysis.io import load_data
from alco_analysis.stats import summary
from pathlib import Path

# df_conc, df_events, df_cities, df_pop = load_data(
#                                           "../data/concession.csv", 
#                                           "../data/events.csv",
#                                           "../data/cities.csv", 
#                                           "../data/rezydenci_2023.xlsx",
#                                           debug=False)
# output_folder = Path("..", "output")
# output_folder.mkdir(parents=True, exist_ok=True)

file_conc   = Path(input("Ścieżka do concession.csv: ").strip())
file_events   = Path(input("Ścieżka do events.csv: ").strip())
file_cities   = Path(input("Ścieżka do cities.csv: ").strip())
file_pop  = Path(input("Ścieżka do rezydenci_20XX.xlsx: ").strip())
output_folder = Path(input("Folder na output: ").strip())

assert file_conc.exists(),  f"Nie znaleziono {file_conc}"
assert file_events.exists(),  f"Nie znaleziono {file_events}"
assert file_cities.exists(),  f"Nie znaleziono {file_cities}"
assert file_pop.exists(),  f"Nie znaleziono {file_pop}"

if file_conc and file_events and file_cities and file_pop:
    df_conc, df_events, df_cities, df_pop = load_data(file_conc,
                                                      file_events,
                                                      file_cities,
                                                      file_pop)

summary(output_folder, df_conc, df_cities, df_events, df_pop)

### Funkcje agregacyjne

In [3]:
from alco_analysis.agg import (
    merge_conc_cities, 
    count_conc_city, 
    count_conc_woj,
    merge_conc_population,
    merge_pop_events)

merged = merge_conc_cities(df_conc, df_cities, debug=True)

# Można zliczyć ilość koncesji w całym dataframe albo tylko dla koncesji 
conc_count_city = count_conc_city(df_conc)
conc_count_woj = count_conc_woj(df_conc)

#Można zrobić merge i sprawdzić ile populacji i koncesji jest w danym rejonie 
df_conc_pop = merge_conc_population(conc_count_woj, df_pop)

#MAlbo zrobić merge i sprawdzić ile populacji i pożarów jest w danym rejonie 
df_pop_eve = merge_pop_events(df_conc, df_events, df_pop)


#Jeśli chcemy sprawdzić położenie to trzeba zmergować z miastami :)
df_all = count_conc_city(merged)

Dopasowane   : 392
Niedopasowane: 33
Nie­użyte     : 0
Koncesje bez lokalizacji przydzielonego miasta
['bienkowka' 'nowy  tomysl' 'bielsko - biala' 'libertow' 'dobron'
 'banska nizna' 'tenczynek' 'stare stracze' 'modlniczka' 'tomaszkowo'
 'dziekanow nowy' 'poronin' 'wegrzce' 'uciechow' 'kokoszkowy' 'siedlecin'
 'weglowice' 'proszowki' 'zascianki' 'srock' 'chelm slaski' 'dys'
 'busko-zdroj' 'plock-biala' 'stary bartkow' 'sekocin nowy' 'miedzychow'
 'zbylitowska gora' 'nowiny' 'gilowice' 'stare proboszczewice' 'wolkowo']

	Dropping 33 rows (7.76% of data)


# Obliczanie korelacji między pożarami a merged (koncesje & lokalizacja)

In [4]:
from alco_analysis.stats import correlation_conc_events

df_corr = correlation_conc_events(df_conc, df_cities, df_events)
df_corr

,zdarzenie,Pearson
0,Pożary (razem) vs koncesja,0.775019
1,P - małe vs koncesja,0.777226
2,P - średnie vs koncesja,0.506427
3,P - duże vs koncesja,0.461134
4,P - bardzo duże vs koncesja,0.298512
5,Miejscowe zagrożenia (razem) vs koncesja,0.806288
6,MZ - małe vs koncesja,0.730022
7,MZ - średnie vs koncesja,0.730979
8,MZ - lokalne vs koncesja,0.810473
9,MZ - katastrofalne vs koncesja,0.214802


In [5]:
from alco_analysis.stats import correlation_conc_pop, correlation_pop_events

corr_conc_pop = correlation_conc_pop(df_conc, df_pop)
print("Korelacja między populacją województwa ilością koncesji",corr_conc_pop)

df_corr.loc[-1] = ["Populacja vs ilość koncesji", corr_conc_pop]
df_corr.index = df_corr.index + 1 
df_corr = df_corr.sort_index() 


corr_pop_ev = correlation_pop_events(df_conc, df_events, df_pop) 
print("Korelacja między populacją województwa a ilością pożarów",corr_conc_pop)

df_corr.loc[-1] = ["Populacja vs liość pożarów (RAZEM)", corr_pop_ev]
df_corr.index = df_corr.index + 1 
df_corr = df_corr.sort_index() 



df_corr.to_csv(output_folder/"corr_notebook.csv")

Korelacja między populacją województwa ilością koncesji 0.8865768607314712
Korelacja między populacją województwa a ilością pożarów 0.8865768607314712


# Heatmapa dla sprzedaży alkoholi

Efekt jest dosyć dramatyczny ale proszę wziąć pod uwagę że to Heatmapa WSZYSTKICH pożarów nawet tych największych które niekoniecznie były spowodowane przez picie alkoholu.

In [6]:
from alco_analysis.heat_map import create_heatmap
heatmap = create_heatmap(df_conc, df_events, df_cities, output_folder)
heatmap

Mapa została zapisana jako 'mapa_koncesji.html'
